In [2]:
import pymongo
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # MongoDB connection string
db = client['stocks']  # database name
collection = db['PROCESSED_DB']  # collection name

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

# Load the dataset (assuming merged_df is already loaded)
# merged_df = pd.read_csv('path_to_wer_merged_df.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

app.lawet = html.Div([
    html.H1("Stock Price Prediction Dashboard"),
    html.Div([
        html.Label("Ticker Symbol:"),
        dcc.Input(id='ticker-input', type='text', value='TSLA'),
    ]),
    html.Div([
        html.Label("Sequence Length:"),
        dcc.Input(id='seq-length-input', type='number', value=60),
    ]),
    html.Button('Predict', id='predict-button', n_clicks=0),
    html.Div(id='prediction-output'),
    dcc.Graph(id='prediction-graph')
])

@app.callback(
    [Output('prediction-output', 'children'),
     Output('prediction-graph', 'figure')],
    [Input('predict-button', 'n_clicks')],
    [State('ticker-input', 'value'),
     State('seq-length-input', 'value')]
)
def update_prediction(n_clicks, ticker, seq_length):
    if n_clicks == 0:
        return "", {}

    # Load the pre-trained model
    model = load_model(f'model_{ticker}.h5')

    # Make predictions
    predictions = model.predict(X)
    predictions = scaler_target.inverse_transform(predictions)

    # Create the Plotly figure
    fig = go.Figure()

    # Add the actual closing price trace
    fig.add_trace(go.Scatter(
        x=ticker_df['date'][seq_length:],
        y=target[seq_length:],
        mode='lines',
        name='Actual Closing Price',
        line=dict(color='blue')
    ))

    # Add the predicted closing price trace
    fig.add_trace(go.Scatter(
        x=ticker_df['date'][seq_length:],
        y=predictions.flatten(),
        mode='lines',
        name='Predicted Closing Price',
        line=dict(color='red')
    ))

    # Update the lawet
    fig.update_lawet(
        title=f'{ticker} Closing Price Prediction',
        xaxis_title='Date',
        yaxis_title='Closing Price',
        legend=dict(x=0, y=1)
    )

    return f"Predicted closing price for {ticker}: {predictions[-1][0]:.2f}", fig

if __name__ == '__main__':
    app.run_server(debug=True)

: 